In [1]:
import numpy as np
import pandas as pd
import itertools
import time
import matplotlib.pyplot as plt

### Definition of all the activation functions ###
def tan_hyp(y) :
    return np.tanh(y)

def softmax(y) :
    return np.exp(y-max(y))/np.sum(np.exp(y-max(y)))

def linear(y) : 
    return y

### Derivatives of the activation functions ###

def der_softmax(y) : 
    a = np.zeros((len(y),len(y)))
    np.fill_diagonal(a,y)
    y = np.reshape(y,(len(y),-1))
    b = np.dot(y,np.transpose(y))
    return a - b

def der_tan_hyp(y) : 
    return (1-y)*(1+y)

def der_linear(y) :
    a = np.zeros((len(y),len(y)))
    np.fill_diagonal(a,y)
    return a

### Definition of loss functions for a single example ###
def mse(y,t) : 
    return 0.5*np.sum((y-t)**2)

def cross_entropy(y,t):
    return -np.sum(t*np.log(y))

def grad_cross_entropy(y,t) :
    return -np.divide(t,y)

def grad_mse(y,t) : 
    return y-t

def forward_pass_customweights(activation_functions,training_sample,w) : 
        a_list = [training_sample]
        h_list = [training_sample]
        output = training_sample
        
        for (activation_function,weight) in zip(activation_functions,w) :
            
            output = np.append(output,np.ones((1,1)),axis = 0)
            output = np.dot(weight,output)
            a_list.append(output)
            
            output = eval(activation_function)(output)
            h_list.append(output)
        return a_list , h_list

In [2]:
###class definition
class neural_network :
    
    def __init__(self, n_hidden,list_nodes,loss_fn,list_activation_functions,learning_rate,momentum) :
        self.n_hidden = n_hidden
        self.list_nodes = list_nodes
        self.loss_fn = loss_fn
        self.list_activation_functions = list_activation_functions
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.weights = []
        self.grad_past = []
        
        for i in range(n_hidden+1) :
            
            np.random.seed(42)
            weight_layer = np.random.randn(list_nodes[i+1],list_nodes[i]+1)
            grad_past_layer = np.zeros((list_nodes[i+1],list_nodes[i]+1))
            self.weights.append(weight_layer)
            self.grad_past.append(grad_past_layer)

    def forward_pass(self,training_sample):
        a_list = [training_sample]
        h_list = [training_sample]
        output = training_sample
        
        for (activation_function,weight) in zip(self.list_activation_functions,self.weights) :
            
            output = np.append(output,np.ones((1,1)),axis = 0)
            output = np.dot(weight,output)
            a_list.append(output)
            
            output = eval(activation_function)(output)
            h_list.append(output)
        return a_list , h_list
    
    def calc_loss(self,calc_output,actual_output) :
        return eval(self.loss_fn)(calc_output,actual_output)
    
    def backward_pass(self,h_list,intermediate_loss,actual_output) : 
        '''implementation of backpropagation'''
        
        der_output_act = eval('der_'+self.list_activation_functions[-1])(h_list[-1])
        grad_lf = eval('grad_'+self.loss_fn)(h_list[-1],actual_output)
        grad_layer = np.dot(der_output_act,grad_lf)
                
        for i in range(len(self.weights)) : 
            
            grad_weight_layer = np.dot(grad_layer,np.transpose(np.append(h_list[len(h_list)-i-2],np.ones((1,1)),axis = 0)))
            
            grad_prev_output = np.dot(np.transpose(self.weights[len(self.weights)-i-1])[:-1,:],grad_layer)
            grad_layer = grad_prev_output * eval('der_'+self.list_activation_functions[len(self.list_activation_functions)-i-2])(h_list[len(h_list)-i-2])
            
            self.weights[len(self.weights)-i-1] = self.gen_delta_rule(self.weights[len(self.weights)-i-1],grad_weight_layer,self.grad_past[len(self.grad_past)-i-1]) 
            self.grad_past[len(self.grad_past)-i-1] = self.learning_rate*grad_weight_layer
            
    def gen_delta_rule(self,weights_present,grad_present,weight_change_previous) :
        return weights_present - (grad_present*self.learning_rate) - (weight_change_previous*self.momentum)
    
    def train(self,input_array,output_array,valid_input_array,valid_output_array) : 
        loss_list = []
        valid_loss_list = []
        epoch_num = 0
        prev_loss_avg = 0
        loss_avg = 0
        
        while (epoch_num<=0 or abs(prev_loss_avg - loss_avg)>=1e-5) : 
            
            prev_loss_avg = loss_avg
            loss_avg = 0
                
            print('Epoch Number : ',epoch_num)
            print('Started')
            print('----------------------------------------')
            for (training_sample,actual_output) in zip(input_array,output_array) :
                
                a_list,h_list = self.forward_pass(training_sample)
                
                intermediate_loss = self.calc_loss(h_list[-1],actual_output)
                
                self.backward_pass(h_list,intermediate_loss,actual_output)
                
                loss_avg = loss_avg + intermediate_loss
            
            loss_avg = loss_avg/len(input_array)
            loss_list.append(loss_avg)
            
            valid_loss_avg = 0
            
            for (training_sample,actual_output) in zip(valid_input_array,valid_output_array) :
                
                a_list,h_list = self.forward_pass(training_sample)
                
                intermediate_loss = self.calc_loss(h_list[-1],actual_output)
                                
                valid_loss_avg = valid_loss_avg + intermediate_loss
            
            valid_loss_avg = valid_loss_avg/len(valid_input_array)
            valid_loss_list.append(valid_loss_avg)
            
            print('Average Loss : ',loss_avg)    
            print('----------------------------------------')
            epoch_num = epoch_num + 1
            
        return loss_list,valid_loss_list
            
    def test(self,input_array,output_array) : 
        accuracy = 0
        for (sample,output) in zip(input_array,output_array) : 
            
            a_list,h_list = self.forward_pass(sample)
            calc_output = h_list[-1]
            
            if np.where(calc_output == np.amax(calc_output)) == np.where(output == np.amax(output)) :
                accuracy = accuracy + 1
            
        accuracy = accuracy / len(input_array)    
        return accuracy 

In [3]:
##### reading the data for the code #####
data = None
input_array = []
output_array = []

test_input_array = []
train_input_array= []

data = pd.read_csv('traingroup16.csv')

for i in range(len(data)) :
    input_array.append(np.array([data['x1'][i],data['x2'][i]]).reshape(2,1))
    if data['label'][i] == 0 :
        output_array.append(np.array([1.0,0.0,0.0]).reshape(3,1))
    if data['label'][i] == 1 :
        output_array.append(np.array([0.0,1.0,0.0]).reshape(3,1))
    if data['label'][i] == 2 :
        output_array.append(np.array([0.0,0.0,1.0]).reshape(3,1))  

test_input_array = input_array[int(np.floor(len(data)*3/4)):]
test_output_array = output_array[int(np.floor(len(data)*3/4)):]

train_input_array = input_array[:int(np.floor(len(data)*3/4))]
train_output_array = output_array[:int(np.floor(len(data)*3/4))]

In [5]:
nn = neural_network(2,[2,5,5,3],'cross_entropy',['tan_hyp','tan_hyp','softmax'],0.001,0.8)

list_avg_error,list_valid_avg_error = nn.train(train_input_array,train_output_array,test_input_array,test_output_array)

accuracy = nn.test(test_input_array,test_output_array)

print("Training Finished")
print("---------------------------------------------")
print("Accuracy : ",accuracy)

Epoch Number :  0
Started
----------------------------------------
Average Loss :  1.4961128115792788
----------------------------------------
Epoch Number :  1
Started
----------------------------------------
Average Loss :  1.298851279446546
----------------------------------------
Epoch Number :  2
Started
----------------------------------------
Average Loss :  1.1756610883804073
----------------------------------------
Epoch Number :  3
Started
----------------------------------------
Average Loss :  1.099848017060967
----------------------------------------
Epoch Number :  4
Started
----------------------------------------
Average Loss :  1.0475561888122795
----------------------------------------
Epoch Number :  5
Started
----------------------------------------
Average Loss :  1.0098978212937963
----------------------------------------
Epoch Number :  6
Started
----------------------------------------
Average Loss :  0.981509646173199
----------------------------------------
Ep

Average Loss :  0.6576066928203511
----------------------------------------
Epoch Number :  58
Started
----------------------------------------
Average Loss :  0.6547918123949416
----------------------------------------
Epoch Number :  59
Started
----------------------------------------
Average Loss :  0.6517522754623538
----------------------------------------
Epoch Number :  60
Started
----------------------------------------
Average Loss :  0.6484746795698071
----------------------------------------
Epoch Number :  61
Started
----------------------------------------
Average Loss :  0.6453286553411267
----------------------------------------
Epoch Number :  62
Started
----------------------------------------
Average Loss :  0.6426043582363271
----------------------------------------
Epoch Number :  63
Started
----------------------------------------
Average Loss :  0.6399047276887206
----------------------------------------
Epoch Number :  64
Started
---------------------------------

Average Loss :  0.48447192908488595
----------------------------------------
Epoch Number :  116
Started
----------------------------------------
Average Loss :  0.48207663936470635
----------------------------------------
Epoch Number :  117
Started
----------------------------------------
Average Loss :  0.48003485104719085
----------------------------------------
Epoch Number :  118
Started
----------------------------------------
Average Loss :  0.4782317330833023
----------------------------------------
Epoch Number :  119
Started
----------------------------------------
Average Loss :  0.4765484249568396
----------------------------------------
Epoch Number :  120
Started
----------------------------------------
Average Loss :  0.4748872812572596
----------------------------------------
Epoch Number :  121
Started
----------------------------------------
Average Loss :  0.47322334691099566
----------------------------------------
Epoch Number :  122
Started
----------------------

Average Loss :  0.40650620256858244
----------------------------------------
Epoch Number :  173
Started
----------------------------------------
Average Loss :  0.40514384498443623
----------------------------------------
Epoch Number :  174
Started
----------------------------------------
Average Loss :  0.40405566707364016
----------------------------------------
Epoch Number :  175
Started
----------------------------------------
Average Loss :  0.40329139575420286
----------------------------------------
Epoch Number :  176
Started
----------------------------------------
Average Loss :  0.40272879598122585
----------------------------------------
Epoch Number :  177
Started
----------------------------------------
Average Loss :  0.4021078252131472
----------------------------------------
Epoch Number :  178
Started
----------------------------------------
Average Loss :  0.4013366269224482
----------------------------------------
Epoch Number :  179
Started
---------------------

Average Loss :  0.37151279127269266
----------------------------------------
Epoch Number :  230
Started
----------------------------------------
Average Loss :  0.3710815750179118
----------------------------------------
Epoch Number :  231
Started
----------------------------------------
Average Loss :  0.37067023393887916
----------------------------------------
Epoch Number :  232
Started
----------------------------------------
Average Loss :  0.37027738040625874
----------------------------------------
Epoch Number :  233
Started
----------------------------------------
Average Loss :  0.3699015930226496
----------------------------------------
Epoch Number :  234
Started
----------------------------------------
Average Loss :  0.3695414531601535
----------------------------------------
Epoch Number :  235
Started
----------------------------------------
Average Loss :  0.3691955789597217
----------------------------------------
Epoch Number :  236
Started
-----------------------

Average Loss :  0.36012636174792156
----------------------------------------
Epoch Number :  288
Started
----------------------------------------
Average Loss :  0.35994496779386587
----------------------------------------
Epoch Number :  289
Started
----------------------------------------
Average Loss :  0.3597629060480223
----------------------------------------
Epoch Number :  290
Started
----------------------------------------
Average Loss :  0.3595802222223223
----------------------------------------
Epoch Number :  291
Started
----------------------------------------
Average Loss :  0.35939695838352703
----------------------------------------
Epoch Number :  292
Started
----------------------------------------
Average Loss :  0.3592131548199669
----------------------------------------
Epoch Number :  293
Started
----------------------------------------
Average Loss :  0.3590288525689793
----------------------------------------
Epoch Number :  294
Started
-----------------------

Average Loss :  0.34922506753360233
----------------------------------------
Epoch Number :  346
Started
----------------------------------------
Average Loss :  0.34905546784115166
----------------------------------------
Epoch Number :  347
Started
----------------------------------------
Average Loss :  0.3488886769982335
----------------------------------------
Epoch Number :  348
Started
----------------------------------------
Average Loss :  0.34872439013843215
----------------------------------------
Epoch Number :  349
Started
----------------------------------------
Average Loss :  0.34856230895384055
----------------------------------------
Epoch Number :  350
Started
----------------------------------------
Average Loss :  0.34840214458315255
----------------------------------------
Epoch Number :  351
Started
----------------------------------------
Average Loss :  0.348243621066086
----------------------------------------
Epoch Number :  352
Started
----------------------